## Maximal Marginal Relevance

MMR (Maximal Marginal Relevance) is a powerful diversity-aware retrieval technique used in information retrieval and RAG pipelines to balance relevance and novelty when selecting documents.

In [3]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import ChatOllama
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain

In [125]:
# TextLoader
loader = TextLoader("langchain_rag_dataset.txt")
raw_documents = loader.load()

# TextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=100,
)

documents = text_splitter.split_documents(raw_documents)

# Adding page numbers to metadata
for i, doc in enumerate(documents):
    doc.metadata['page'] = i + 1

# Embeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# FAISS
vectorstore = FAISS.from_documents(
    documents,
    embeddings
)

# Retrieval
retriever = vectorstore.as_retriever(
    search_kwargs={
        "k": 3
    },
    search_type="mmr"
)

# Model 
llm = ChatOllama(model="gemma3:4b-it-q4_K_M", temperature=0.2)

In [129]:
from langchain_core.output_parsers import StrOutputParser

# So werden einzelne Dokumente mit Metadaten formatiert
document_prompt = PromptTemplate.from_template("""  
Page number: {page}                                                                                         
Source: {source}
Content: {page_content}
""")

prompt = PromptTemplate.from_template("""
You are a helpful assistant. You will be given a question and you will answer it based on the context provided in the documents.
If the question is not answered by the context, you can say "I don't knwow" or "I'm sorry, I don't know that.

Each document has metadata. Use them to provide a better answer.

Return your answer in markdown format.

Context:
{context}

Return the metadata as to your answer as follows:

Metadata:
*page
*source

Question: {input}
""")

# Document Chain
document_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=prompt,
    document_prompt=document_prompt
)

# Retrieval Chain
retrieval_chain = create_retrieval_chain(
    retriever=retriever,
    combine_docs_chain=document_chain
)

In [130]:
from IPython.display import Markdown, display

query = {"input": "How does LangChain support agents and memory?"}
response = retrieval_chain.invoke(query)

display(Markdown(response['answer']))

Metadata:
*page: 1
*source: langchain_rag_dataset.txt

LangChain provides abstractions for working with prompts, chains, memory, and agents, making it easier to build complex LLM-based systems. Memory in LangChain helps models retain previous interactions, making multi-turn conversations more coherent. Agents in LangChain can use tools like calculators, search APIs, or custom functions based on the instructions they receive.

In [131]:
response

{'input': 'How does LangChain support agents and memory?',
 'context': [Document(id='e88bb1b9-51dd-4186-a70b-4c3610a42bde', metadata={'source': 'langchain_rag_dataset.txt', 'page': 3}, page_content='Memory in LangChain helps models retain previous interactions, making multi-turn conversations more coherent.\nAgents in LangChain can use tools like calculators, search APIs, or custom functions based on the instructions they receive.'),
  Document(id='6897db18-4e79-40a8-a0b2-9584e060ed0d', metadata={'source': 'langchain_rag_dataset.txt', 'page': 5}, page_content='Chroma is a lightweight vector store often used in LangChain for embedding-based document storage and retrieval.\nPrompt templates in LangChain support Jinja-style formatting and variable injection to customize model inputs.'),
  Document(id='15214e69-f4a6-4599-8f6f-2e6c0d3e9550', metadata={'source': 'langchain_rag_dataset.txt', 'page': 1}, page_content='LangChain is an open-source framework designed to simplify the development o